# FQA日本語クリーナー for `*.jsonl`

## 事前準備

In [15]:
import pandas as pd
import numpy as np
import os
import shutil
import re
import json

### 入力ファイルと出力ファイルの指定

In [16]:
file_paths = ["test_data.jsonl"]
output_file = "processed_data"

### 質問に相当するkey, 回答に相当するkeyの指定

In [17]:
target_q = 'prompt'
target_a = 'completion'

### ファイルの読み込みとPandas DataFrameの作成

複数のJSONL（JSON Lines）形式のファイルからデータを読み込み、それらを一つのデータフレームにまとめるために使用されます

In [18]:
def read_json_lines(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                # バックスラッシュをエスケープ
                line = re.sub(r'\\', r'\\\\', line)
                json_line = json.loads(line.strip())  # 余分な空白を除去
                data.append(json_line)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {file_path} at line {line_number}: {line}")
                print(f"Error message: {e}")
    return data

def load_files_to_dataframe(file_paths):
    all_data = []
    for file_path in file_paths:
        file_data = read_json_lines(file_path)
        all_data.extend(file_data)
    df = pd.DataFrame(all_data, columns=[target_q, target_a])
    return df

# DataFrameに読み込む
df = load_files_to_dataframe(file_paths)

## 日本語修正

### カスタム関数

In [19]:
# 正規表現パターンを定義して、行末の全角スペースと半角スペースを削除
def remove_trailing_spaces(text):
    return re.sub(r'[ \u3000]+$', '', text)

# 例:
# - 変換前:【カテゴリ】メッセージ：「メッセージ内容」の形式を
# - 変換後:【カテゴリ】カテゴリのメッセージとして「メッセージ内容」を得ました。具体的にどうすればよいでしょうか？に変換する関数
def custom_transform1(text):
    # 正規表現パターンを定義
    pattern = r'【(.+?)】メッセージ：(?:「(.+?)」|｢(.+?)｣)'
    
    # 正規表現で一致する部分を抽出
    match = re.match(pattern, text)
    
    if match:
        category = match.group(1)
        message_content = match.group(2)
        
        # 新しい形式に変換
        new_message = f"【{category}】{category}のメッセージとして「{message_content}」を得ました。具体的にどうすればよいでしょうか？"
        return new_message
    else:
        return text

# 例: 
# - 変換前: 分析方法(パターン認識) 
# - 変換後: 分析方法(パターン認識)について教えてほしい。
def custom_transform2(text):
    # 正規表現パターンを定義
    pattern = r'(.*方法)\((.*)\)'
    
    # 正規表現で一致する部分を抽出して置換
    match = re.match(pattern, text)
    
    if match:
        sentence = match.group(1)
        supplement = match.group(2)
        
        # 新しい形式に変換
        new_sentence = f"{sentence}({supplement})について教えてほしい。"
        return new_sentence
    else:
        return text

# 例: 
# - 変換前: データ処理方法（このデータの前処理）
# - 変換後: データ処理方法（このデータの前処理）について教えてほしい。
def custom_transform3(text):
    # 正規表現パターンを定義
    pattern = r'(.*)[\(\（](.*)[\)\）]'
    
    # 正規表現で一致する部分を抽出して置換
    match = re.match(pattern, text)
    
    if match:
        sentence = match.group(1)
        message = match.group(2)
        
        # 新しい形式に変換
        new_sentence = f"{sentence}({message})について教えてほしい。"
        return new_sentence
    else:
        return text

# 例
# - 変換前:【技術】新しいアルゴリズムの開発方法について教えてください。
# - 変換後: 技術に関する質問です。新しいアルゴリズムの開発方法について教えてください。
def custom_transform4(text):
    # 正規表現パターンを定義
    pattern = r'【(.+?)】(.+)'
    
    # 正規表現で一致する部分を抽出
    match = re.match(pattern, text)
    
    if match:
        category = match.group(1)
        question = match.group(2)
        
        # 新しい形式に変換
        new_text = f"{category}に関する質問です。{question}"
        return new_text
    else:
        return text

# 行末が "。" もしくは "？" で終わっていない行を抽出して更新する関数 (確認用に使用)
# - 変換前: このプロジェクトの詳細
# - 変換後: このプロジェクトの詳細について教えてください。
def append_text_if_no_punctuation(text):
    if not re.search(r'[。？]$', text):  # 行末が "。" もしくは "？" で終わっていないか確認
        return text + "について教えてください。"
    return text


# 空行とスペースを削除する関数
def remove_empty_lines_and_spaces(text):
    text = re.sub(r'[A-Z][0-9]{4}', '', text)
    text = re.sub(r'[A-Z][0-9]{3}', '', text)
    text = re.sub(r'\n\s*\n', '\n', text)  # 空行を削除
    text = text.replace(' ', '')  # 半角スペースを削除
    text = text.replace('　', '')  # 全角スペースを削除
    #text = text.replace('"', '')
    text = re.sub(r'[\r\n]+', '', text)  # 改行コードを削除
    return text

### 処理1: カスタム処理 (処理の順番の依存関係に注意)

In [20]:
df[target_q] = df[target_q].str.replace('。$', '', regex=True)
df[target_q] = df[target_q].apply(remove_trailing_spaces)
#df[target_q] = df[target_q].apply(custom_transform1)
df[target_q] = df[target_q].apply(custom_transform3)
df[target_q] = df[target_q].apply(custom_transform4)

### 処理2: 簡単な正規表現と置換のみで対応可能 (処理の順番の依存関係に注意)

In [21]:
df[target_q] = df[target_q].str.replace(r'です(\。)?$', '', regex=True)
df[target_q] = df[target_q].str.replace(r'か(\。)?$', 'か？', regex=True)
df[target_q] = df[target_q].str.replace(r'は(\。)?$', 'は？', regex=True)
df[target_q] = df[target_q].str.replace(r'出る$', 'でる？', regex=True)
df[target_q] = df[target_q].str.replace(r'出ます$', 'でます？', regex=True)
df[target_q] = df[target_q].str.replace('エラーがでます$', 'エラーがでる。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('エラーがでました$', 'エラーがでる。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('エラーがでる$', 'エラーがでる。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('エラーがでた$', 'エラーがでた。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('エラーになります$', 'エラーになります。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('エラーになりました$', 'エラーになりました。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('メッセージがでる$', 'メッセージがでる。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('メッセージがでます$', 'メッセージがでます。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('お願いします$', 'お願いしたいのですがどうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('消えてまう$', '消えてしまいます。どうすれば良いですか？', regex=True)
df[target_q] = df[target_q].str.replace('ない$', 'ない。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('たい$', 'たい。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('方法$', '方法を教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('意味$', '意味について教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('設定$', '設定について教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('数$', '数について教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('制限$', '制限について教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('感じる$', '感じる。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('なる$', 'なることについて教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('しまう$', 'しまうことについて教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('ついて$', 'ついて教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('された$', 'された。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('されます$', 'されます。どうすれば良いですか？', regex=True)
df[target_q] = df[target_q].str.replace('される$', 'される。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('されます$', 'されます。どうすれば良いですか？', regex=True)
df[target_q] = df[target_q].str.replace('ている$', 'ている。どうすれば良いですか？', regex=True)
df[target_q] = df[target_q].str.replace('所$', '所について教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('方$', '方について教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('する$', 'する。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('しまった$', 'しまった。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('届いた$', '届いた。どうすれば良いか？。', regex=True)
df[target_q] = df[target_q].str.replace('違い$', '違いについて教えてほしい。', regex=True)
df[target_q] = df[target_q].str.replace('悪い$', '悪い。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('ある$', 'ある。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('変わる$', '変わる。どうすれば良いか？', regex=True)
df[target_q] = df[target_q].str.replace('した$', 'した。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('なった$', 'なった。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('遅い$', '遅い。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('取り扱い$', '取り扱いについて教えてください？', regex=True)
df[target_q] = df[target_q].str.replace('ほしい$', 'ほしい。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('ありません$', 'ありません。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('生じる$', '生じる。どうしたら良いか？', regex=True)
df[target_q] = df[target_q].str.replace('注意点$', '注意点について教えてください。', regex=True)
df[target_q] = df[target_q].str.replace('したいです$', 'したいです。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('開けません$', '開けません。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('されている$', 'されている。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('されています$', 'されています。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('います$', 'います。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('ません$', 'ません。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('なります$', 'なります。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('あります$', 'あります。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('きます$', 'きます。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('します$', 'します。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('なかった$', 'なかった。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('かかった$', 'かかった。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('かかります$', 'かかります。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('かかる$', 'かかる。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('られる$', 'られる。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('られます$', 'られます。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('れた$', 'れた。どうすればよいですか？', regex=True)
df[target_q] = df[target_q].str.replace('がいる$', 'がいる。どうすればよいですか？', regex=True)

### ぬけもれ確認用

- 基本的には、末尾が"。"か"?"で必ず終わるようにしている。
- そのため、`df[target_q]`の末尾が "。" もしくは "？" で終わっていない行を抽出し、ぬけもれを調べている

In [22]:
df[target_q] = df[target_q].apply(append_text_if_no_punctuation)

### 質問, 回答も含めて空行やスペースを削除

In [23]:
# 全てのセルに対して空行を削除
df = df.applymap(remove_empty_lines_and_spaces)

## ファイルの保存

### onelinerのcsvとして保存(2列)

In [24]:
df.to_csv(f'{output_file}.csv', index=False)

### onelinerのtxtとして保存

In [25]:
# 質問と回答を結合してテキストデータを作成
merged_text = df[target_q] + df[target_q]
# テキストファイルに保存
with open(f'{output_file}.txt', 'w') as file:
    for text in merged_text:
        file.write(text + "\n")

### 質問の行末確認用コード

- `filtered_output.csv`というファイルを作成して、行末の状態を確認する
- 特にQuestionであれば"?"で終わっていない文章を抽出することがそもそもの目的

In [26]:
#df = pd.DataFrame(data)

# "。"もしくは"？"で終わっていない行を抽出
pattern = r'[^。？? 教えてください]$'
filtered_df = df[df[target_q].str.contains(pattern, na=False)]

# 結果をファイルに出力
filtered_df.to_csv('filtered_output.csv', index=False, encoding='utf-8-sig')

print("Filtered data has been written to 'filtered_output.csv'")

Filtered data has been written to 'filtered_output.csv'


### 処理後の結果をjsonで出力

In [27]:
def save_dataframe_to_jsonl(df, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        for _, row in df.iterrows():
            # JSON文字列内のバックスラッシュと改行を元の状態に戻す
            prompt = row['prompt'].replace('\\\\', '\\').replace('\\n', '\n')
            completion = row['completion'].replace('\\\\', '\\').replace('\\n', '\n')
            json_line = json.dumps({"prompt": prompt, "completion": completion}, ensure_ascii=False)
            file.write(json_line + '\n')

In [28]:
save_dataframe_to_jsonl(df, f'{output_file}.jsonl')